In [1]:
import pandas as pd
import numpy as np
import nltk
import re
import math
from functools import reduce
from unicodedata import normalize

In [2]:
dados = pd.read_csv('data/estadao_noticias_eleicao.csv')
dados = dados.replace(np.nan, '', regex=True)

# Join do conteúdo
Juntando os títulos das notícias com seus respectivos conteúdos,
para posteriomente facilitar a tokenização

In [3]:
def limpar_texto(texto):
    pattern = re.compile('[^a-zA-Z0-9 ]')
    texto = normalize('NFKD', texto).encode('ASCII', 'ignore').decode('ASCII')
    return pattern.sub(' ', texto)

In [4]:
materias = dados.titulo + " " + dados.subTitulo +  " " + dados.conteudo
materias = materias.apply(lambda texto: "" if isinstance(texto, float) else limpar_texto(texto))
ids = dados.idNoticia
    

# Tokenizando conteúdo
Criando tokens com cada palavra do texto para que posteriormente possam ser indexadas e associadas aos respectivos ids das notícias

In [5]:
tokens = materias.apply(nltk.word_tokenize)

# Indexando tokens
Criando indices invertidos com os tokens para poder aplicar os métodos de busca 

In [6]:
index = {}

for i in range(len(tokens)):
    id_noticia = ids[i]
    palavras = tokens[i]
    for palavra in palavras:
        palavra = palavra.lower()
        if palavra not in index:
            index[palavra] = []
        
        id_rec = reduce(lambda found, id_not: id_not if id_not[1] == id_noticia else found, index.get(palavra, []), None)
        
        if not id_rec:
            docs = index[palavra]
            docs.append((1, id_noticia))
        else:
            docs = index[palavra]
            index_id = docs.index(id_rec)
            docs[index_id] = (id_rec[0] + 1, id_rec[1])

In [7]:
def gera_tf_vetor(frase):
    termos = frase.split(" ")
    doc_tf = {}
    
    for i in range(len(termos)):
        termo = termos[i]
        docs = index[termo]
        for doc in docs:
            doc_id = doc[1]
            tf = doc[0]
            
            if doc_id not in doc_tf:
                doc_tf[doc_id] = np.array([0 if j != i else tf for j in range(len(termos))])
            else:
                doc_vector = doc_tf[doc_id]
                doc_tf[doc_id] = np.array([doc_vector[j] if j != i else tf for j in range(len(termos))])
        
    return doc_tf

def gera_idf_vetor(frase):
    termos = frase.split(" ")
    idf_vector = np.array([math.log((len(materias)+1)/len(index[termo])) for termo in termos])
    return idf_vector

def gera_query_vetor(frase):
    termos = frase.split(" ")
    vetor = np.array([1 if index.get(termo) else 0 for termo in termos])
    return vetor

In [8]:
def buscar_por_tf(frase):
    docs_tf = gera_tf_vetor(frase)
    query = gera_query_vetor(frase)
    
    doc_rank = sorted(list(docs_tf.items()), key=lambda doc: np.dot(doc[1], query), reverse=True)[:5] 
    return [doc[0] for doc in doc_rank]

In [9]:
def buscar_por_tf_idf(frase):
    docs_tf = gera_tf_vetor(frase)
    idf = gera_idf_vetor(frase)
    
    doc_rank = sorted(list(docs_tf.items()), key=lambda doc: np.dot(doc[1], idf), reverse=True)[:5]
    return [doc[0] for doc in doc_rank]

In [18]:
# print(gera_idf_vetor('lula preso'))
# print(gera_tf_vetor('lula preso'))
# print(len(materias))
# print(len(index['preso']))
# print(len(materias)/len(index['preso']))
print(buscar_por_tf('projeto de lei'))
print(buscar_por_tf_idf('segundo turno'))
# gera_tf_vetor('segundo turno')
# rec = reduce(lambda found, id_not: id_not if id_not[1] == 7 else found, index.get('turno', []), None)
# print(rec)

[7, 155, 6554, 3942, 7017]
[2744, 2112, 7672, 1235, 2388]
